#### This task involves scraping social media websites to get the follower count for data analysis. The social media websites that follow a similar process as the one below are Facebook, TikTok, and YouTube. Here, we will be using Facebook as an example. 

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import date

1. We start off by retrieving the URLs of the profiles that we need to get the follower count from.

In [ ]:
facebook = pd.read_excel('profiles.xlsx', sheet_name = 'Facebook')

#creates tuples (handle, url, status)...
urlsTuple = list(zip(facebook['Status'],facebook['Handle'], facebook['URL']))

#list(a) creates a list in lists [handle, url]...
urls = [list(a) for a in urlsTuple] 

totalCount = len(urls)
count = 1

2. We can now retrieve the like count for the respective profiles on Facebook. 

In [ ]:
browser = webdriver.Firefox()

for url in urls:
    print(f"[{count}/{totalCount}] Retrieving subscribers for {url[1]}...")

    if url[0] == 'Inactive':
        url.append(0)

    else:       
        browser.get(url[2])
        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, 'html.parser')

        description = soup.find('meta', attrs={'name':'description'})

        # Content is a string of form "... Singapore. XXXXX likes....."
        content = description['content']

        # Locate like count between words Singapore and likes
        likesString = content[content.find('. ') + 2:content.find(' likes')]

        if likesString.endswith("K"):
            url.append(int(float(likesString[:-1]) * 1000))

        elif likesString.endswith("M"):
            url.append(int(float(likeString[:-1]) * 1000000))

        else: 
            # Split numbers using ',' --> get list of number parts as strings
            # eg. '3,370,140' --> ['3', '370', '140']
            likesList = likesString.split(',')

            # Define likes as a blank before combining elements in the list
            likes = ''

            # Combine elements in the list
            for x in likesList:
                likes += x
            likes = int(likes)
            url.append(likes)

    count += 1

browser.close()

3. Lastly, we will generate an output file containing the like count for each profile.

In [ ]:
fileName = f"facebook_likes_{date.today()}.xlsx"
df=pd.DataFrame(urls, columns=['Status','Handle', 'URL', 'Likes'])
df.to_excel(fileName, index=False)